# ID3 Algorithm P2

So far we've covered decision trees growing with an entropy criterion. In doing so, however, we glossed over how that would actually work. In this assignment we'll give more detail into how an algorithm to do that would practically function.

Here we'll cover one popular algorithm for building a decision tree using entropy and information gain: **ID3**.

# Calculating Entropy

Since this algorithm is based on entropy let's go over a quick example of how to calculate it.

Let's say we have 20 students, and we're trying to classify them as male and female. The only attribute we have is whether their height is tall, medium, or short. Of the 20 students, 12 are boys with and 8 are girls. Of the 12 boys, 4 are tall, 6 are medium and 2 are short. Of the 8 girls, 1 is tall, 2 are medium, and 5 are short.

In [4]:
# Calculate tall or medium
from math import log2

# Tall
H_tall = 4/5 * log2(5/4) + 1/5 * log2(5/1)
H_not_tall = 8/15 * log2(15/8) + 7/15 * log2(15/7)

Entropy_tall = 5/20 * H_tall + 15/20 * H_not_tall

# Medium
H_med = 6/8 * log2(8/6) + 2/8 * log2(8/2)
H_not_med = 6/12 * log2(12/6) + 6/12 * log2(12/6)

Entropy_med = 8/20 * H_med + 12/20 * H_not_med

print(str(Entropy_tall) + '\n' + str(Entropy_med))


0.9280757477080679
0.9245112497836532


# Pseudocoding the Algorithm

Pseudocode is the processes of writing the steps and logic you would implement in code, but in normal language rather than commands a programming language could execute. It can be a useful way to chart out how you want to build an algorithm and is a common topic for technical interviews. Here we'll use pseudocode to explain the ID3 algorithm.

Here is reasonable pseudocode for ID3, which we'll then follow up with an explanation of the steps. The outcome for this variable will be A or B. An attribute is denoted ai. A value of that attribute is vi.

Algorithm(Observations, Outcome, Attributes)
    - Create a root node.
    - If all observations are 'A', label root node 'A' and return.
    - If all observations are 'B', label root node 'B' and return.
    - If no attributes return the root note labeled with the most common Outcome.
Otherwise, start:
    - For each value vi of each attribute ai, calculate the entropy.
    - The attribute ai and value vi with the lowest entropy is the best rule.
    - The attribute for this node is then ai
            - Split the tree to below based on the rule ai = vi
            - Observations vi is the subset of observations with value vi
            - If Observationsvi is empty cap with node labeled with most common Outcome
            - Else at the new node start a subtree (Observationsvi, Target Outcome, Attributes - {ai}) and repeat the algorithm

Now let's walk through this pseudocode algorithm in plain English piece by piece.

First you create a root node. Simple enough, you have to start with something.

The next two lines say that if you're already exclusively one class, just label with that class and you're done. Similarly the following line says that if you don't have any attributes left you're also done, labeling with the most common outcome.

Then we get into the real algorithm. First you have to find the best attribute by calculating entropies for all possible values. The attribute value pair with the lowest entropy is then the best attribute and the attribute you give to the node.

You use that rule to split the node, creating subsets of observations. There are then two new nodes, each with a subset of the observations corresponding to the rule. If obsevations are null then label with the dominant outcome.

Otherwise at each new node start the algorithm again.

This is how a decision tree would actually function. Understanding this should give you some insight into how this algorithm works and reveals several attributes of the algorithm. Firstly, the solution is not necessarily optimal. The tree can get stuck in local optima, just like with the gradient descent algorithm. It also has no way to work backwards if it finds itself in an informationless space. You can add criteria to make it stop before the tree has grown to run out of attributes or all leaf nodes are single classes.

# DRILL:

Look over the code for this real ID3 Algorithm in python. Note how well the author breaks up functionality into individual, reusable, well-named helper functions. See if you can match our pseudocode steps to the code in this example.

In [2]:
import math

#find item in a list
def find(item, list):
    for i in list:
        if item(i): 
            return True
        else:
            return False

#find most common value for an attribute
# - Create root node.
def majority(attributes, data, target):
    #find target attribute
    valFreq = {}
    #find target in data
    index = attributes.index(target)
    #calculate frequency of values in target attr
    # - if all observations are a, return, or b.
    for tuple in data:
        if (valFreq.has_key(tuple[index])):
            valFreq[tuple[index]] += 1 
        else:
            valFreq[tuple[index]] = 1
    max = 0
    major = ""
    # - If no attributes return the root note labeled with the most common Outcome.
    for key in valFreq.keys():
        if valFreq[key]>max:
            max = valFreq[key]
            major = key
    return major

#Calculates the entropy of the given data set for the target attr
# - For each value vi of each attribute ai, calculate the entropy.
def entropy(attributes, data, targetAttr):

    valFreq = {}
    dataEntropy = 0.0
    
    #find index of the target attribute
    i = 0
    for entry in attributes:
        if (targetAttr == entry):
            break
        ++i
    
    # Calculate the frequency of each of the values in the target attr
    for entry in data:
        if (valFreq.has_key(entry[i])):
            valFreq[entry[i]] += 1.0
        else:
            valFreq[entry[i]]  = 1.0

    # Calculate the entropy of the data for the target attr
    for freq in valFreq.values():
        dataEntropy += (-freq/len(data)) * math.log(freq/len(data), 2) 
        
    return dataEntropy

def gain(attributes, data, attr, targetAttr):
    """
    Calculates the information gain (reduction in entropy) that would
    result by splitting the data on the chosen attribute (attr).
    """
    valFreq = {}
    subsetEntropy = 0.0
    
    #find index of the attribute
    i = attributes.index(attr)

    # Calculate the frequency of each of the values in the target attribute
    for entry in data:
        if (valFreq.has_key(entry[i])):
            valFreq[entry[i]] += 1.0
        else:
            valFreq[entry[i]]  = 1.0
    # Calculate the sum of the entropy for each subset of records weighted
    # by their probability of occuring in the training set.
    for val in valFreq.keys():
        valProb        = valFreq[val] / sum(valFreq.values())
        dataSubset     = [entry for entry in data if entry[i] == val]
        subsetEntropy += valProb * entropy(attributes, dataSubset, targetAttr)

    # Subtract the entropy of the chosen attribute from the entropy of the
    # whole data set with respect to the target attribute (and return it)
    return (entropy(attributes, data, targetAttr) - subsetEntropy)

#choose best attibute 
# - The attribute ai and value vi with the lowest entropy is the best rule.
def chooseAttr(data, attributes, target):
    best = attributes[0]
    maxGain = 0;
    for attr in attributes:
        newGain = gain(attributes, data, attr, target) 
        if newGain>maxGain:
            maxGain = newGain
            best = attr
    return best

#get values in the column of the given attribute 
def getValues(data, attributes, attr):
    index = attributes.index(attr)
    values = []
    for entry in data:
        if entry[index] not in values:
            values.append(entry[index])
    return values

def getExamples(data, attributes, best, val):
    examples = [[]]
    index = attributes.index(best)
    for entry in data:
        #find entries with the give value
        if (entry[index] == val):
            newEntry = []
            #add value if it is not in best column
            for i in range(0,len(entry)):
                if(i != index):
                    newEntry.append(entry[i])
            examples.append(newEntry)
    examples.remove([])
    return examples

def makeTree(data, attributes, target, recursion):
    # - The attribute for this node is then ai
    recursion += 1
    #Returns a new decision tree based on the examples given.
    # - Split the tree to below based on the rule ai = vi
    data = data[:]
    vals = [record[attributes.index(target)] for record in data]
    default = majority(attributes, data, target)

    # If the dataset is empty or the attributes list is empty, return the
    # default value. When checking the attributes list for emptiness, we
    # need to subtract 1 to account for the target attribute.
    # - Observations vi is the subset of observations with value vi
    # - If Observationsvi is empty cap with node labeled with most common Outcome
    if not data or (len(attributes) - 1) <= 0:
        return default
    # If all the records in the dataset have the same classification,
    # return that classification.
    elif vals.count(vals[0]) == len(vals):
        return vals[0]
    else:
        # Choose the next best attribute to best classify our data
        best = chooseAttr(data, attributes, target)
        # Create a new decision tree/node with the best attribute and an empty
        # dictionary object--we'll fill that up next.
        tree = {best:{}}
    
        # Create a new decision tree/sub-node for each of the values in the
        # best attribute field
        # - Else at the new node start a subtree (Observationsvi, Target Outcome, Attributes - {ai}) and repeat the algorithm
        for val in getValues(data, attributes, best):
            # Create a subtree for the current value under the "best" field
            examples = getExamples(data, attributes, best, val)
            newAttr = attributes[:]
            newAttr.remove(best)
            subtree = makeTree(examples, newAttr, target, recursion)
    
            # Add the new subtree to the empty dictionary object in our new
            # tree/node we just created.
            tree[best][val] = subtree
    
    return tree